In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from activ import load_data
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from activ.data_normalization import data_normalization
from scipy.spatial.distance import pdist
from sklearn.model_selection import KFold
import pandas as pd
from activ.readfile import TrackTBIFile

/Users/ahyeon/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
subdata = load_data()
sub_oc_features = subdata.outcome_features
sub_bm_features = subdata.biomarker_features
sub_oc = subdata.outcomes
sub_bm = subdata.biomarkers

In [14]:
fname = '/Users/ahyeon/data/activ/rawdata.h5'
TrackTBIFile.write(fname, sub_bm, sub_oc)

In [7]:
sub_bm[:,113]

array([15., 14., 15., 15., 15., 15., 15., 15.,  7., 15., 15., 14., 15.,
       15., 15., 14., 15., 14.,  3., 15., 15., 15.,  3., 15., 15., 15.,
       15.,  3.,  3., 15., 15., 15., 14.,  3., 15., 15., 15., 15., 15.,
       15., 15.,  3.,  3., 15., 15.,  3., 15.,  3.,  3., 15., 15.,  3.,
       11., 15., 15., 15., 15., 15., 15., 15., 15.,  3., 15.,  3.,  3.,
        3., 15., 15.,  3.,  3., 15.,  9., 15., 15., 15., 15., 15., 15.,
       15., 15., 11., 14., 15., 15., 14., 14., 15., 15., 15., 15., 13.,
       12., 15., 15., 15., 15., 15., 15., 15., 15., 15., 14., 15., 14.,
       14., 14., 15., 14., 15., 15., 15., 15., 14., 15., 15., 13., 15.,
       14., 15., 14., 15., 14., 15., 15., 15., 14., 15., 15., 15., 13.,
       14., 15., 14., 15., 15., 11.,  3., 15., 15., 15., 15.,  9., 15.,
       14., 15., 14., 15., 15., 15.,  9.,  6., 15.,  7., 15., 14., 15.,
       14., 15., 15., 15., 14., 11., 15., 14., 14., 15., 15., 14., 11.,
       15., 15., 15., 15., 15., 15., 15., 15., 14., 15., 15.,  9

In [3]:
# mild
count = 0
patients = []
for i in range(247):
    if sub_bm[:,113][i] >= 13.0 and sub_bm[:,113][i] <= 15.0:
        patients.append(i)
        count+=1
        
bm_gcs = np.zeros((len(patients), sub_bm.shape[1]))
oc_gcs = np.zeros((len(patients), sub_oc.shape[1]))
for ii,p in enumerate(patients):
    bm_gcs[ii,:] = sub_bm[p,:]
    oc_gcs[ii,:] = sub_oc[p,:]
print(bm_gcs.shape)
print(oc_gcs.shape)

(210, 235)
(210, 177)


In [4]:
bm_gcs[:,113]

array([15., 14., 15., 15., 15., 15., 15., 15., 15., 15., 14., 15., 15.,
       15., 14., 15., 14., 15., 15., 15., 15., 15., 15., 15., 15., 15.,
       15., 14., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15.,
       15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15., 15.,
       15., 15., 15., 15., 15., 15., 15., 15., 14., 15., 15., 14., 14.,
       15., 15., 15., 15., 13., 15., 15., 15., 15., 15., 15., 15., 15.,
       15., 14., 15., 14., 14., 14., 15., 14., 15., 15., 15., 15., 14.,
       15., 15., 13., 15., 14., 15., 14., 15., 14., 15., 15., 15., 14.,
       15., 15., 15., 13., 14., 15., 14., 15., 15., 15., 15., 15., 15.,
       15., 14., 15., 14., 15., 15., 15., 15., 15., 14., 15., 14., 15.,
       15., 15., 14., 15., 14., 14., 15., 15., 14., 15., 15., 15., 15.,
       15., 15., 15., 15., 14., 15., 15., 15., 14., 15., 15., 15., 14.,
       14., 15., 14., 15., 15., 13., 15., 15., 13., 15., 15., 15., 15.,
       14., 15., 14., 15., 15., 15., 15., 15., 15., 15., 15., 15

In [9]:
# moderate
count = 0
patients = []
for i in range(247):
    if sub_bm[:,113][i] >= 9.0 and sub_bm[:,113][i] <= 12.0:
        patients.append(i)
        count+=1
        
bm_gcs_mod = np.zeros((len(patients), sub_bm.shape[1]))
oc_gcs_mod = np.zeros((len(patients), sub_oc.shape[1]))
for ii,p in enumerate(patients):
    bm_gcs_mod[ii,:] = sub_bm[p,:]
    oc_gcs_mod[ii,:] = sub_oc[p,:]
print(bm_gcs_mod.shape)
print(oc_gcs_mod.shape)

TrackTBIFile.write('/Users/ahyeon/data/activ/gcs_mod.h5', bm_gcs_mod, oc_gcs_mod)

(13, 235)
(13, 177)


247

In [10]:
# severe
count = 0
patients = []
for i in range(247):
    if sub_bm[:,113][i] <= 8.0:
        patients.append(i)
        count+=1
        
bm_gcs_sev = np.zeros((len(patients), sub_bm.shape[1]))
oc_gcs_sev = np.zeros((len(patients), sub_oc.shape[1]))
for ii,p in enumerate(patients):
    bm_gcs[ii,:] = sub_bm[p,:]
    oc_gcs[ii,:] = sub_oc[p,:]

TrackTBIFile.write('/Users/ahyeon/data/activ/gcs_sev.h5', bm_gcs_sev, oc_gcs_sev)

In [12]:
fname = '/Users/ahyeon/data/activ/gcs13-15/gcs_13-15.h5'
TrackTBIFile.write(fname, bm_gcs, oc_gcs)

In [ ]:
raw = load_data()
labels = raw.outcome_features
nmfdata = load_data(uoinmf=True)
rawbm = raw.biomarkers #(247,235)
feature_names = raw.outcome_features
A = data_normalization.data_normalization(raw.outcomes, 'positive') #(247,177)
w = nmfdata.outcomes #(247,6)
h = nmfdata.outcome_bases #(6,177)

In [ ]:
def drop_x_percent(A,w,h,feature_names,percent):
    total_features = h.shape[1]
    num_features_to_drop = np.round(total_features*percent//100)
    h_sum = np.sum(h, axis=0)
    h_sum_sorted = np.argsort(h_sum)
    indices_lowest_values = h_sum_sorted[:num_features_to_drop]
    dropped_features = sorted(feature_names[indices_lowest_values])
    h_new = np.delete(h, indices_lowest_values, axis=1)
    A_new = np.delete(A, indices_lowest_values, axis=1) 
    return A_new, h_new, dropped_features

In [ ]:
percent = [0,10,20,30,40,50,60,70,80,90]
for p in percent:
    A_features_dropped,h_new, dropped_features = drop_x_percent(A,w,h,feature_names,p)
    fname = '/Users/ahyeon/data/activ/gcs13-15/bm_gcs_13-15.h5'.format(p)
    TrackTBIFile.write(fname, sub_bm, A_features_dropped)

In [23]:
help(TrackTBIFile)

Help on class TrackTBIFile in module activ.readfile:

class TrackTBIFile(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, filename)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  write(dest, biomarkers, outcomes, biomarker_features=None, outcome_features=None, patient_ids=None) from builtins.type
 |      Write biomarkers and outcomes to an HDF5 file
 |      
 |      Args:
 |          dest                : the path to an HDF5 file or the :class:`h5py.Group` to write to
 |          biomarkers          : the biomarker data with shape (n_samples, n_features)
 |          outcomes            : the outcomes data with shape (n_samples, n_features)
 |          biomarker_features  : the names of the biomarker features (optional)
 |          outcome_features    : the names of the outcome features (optional)
 |          patient_ids      